In [1]:
%load_ext autoreload
%autoreload 2

# Load parcels (V2.1.3)

Parcels might need to be downloaded locally to run the coupled version

In [2]:
MODULE_PATH = '/home/ursho/PhD/Projects/Pteropods/My_parcels/Parcels_master_copy/parcels/parcels/__init__.py'
MODULE_NAME = "parcels"
import importlib
import sys
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

# Import all modules

In [3]:
import os
import logging
import datetime
import xarray as xr
from parcels import ParticleSet

sys.path.insert(1,"/net/kryo/work/ursho/PhD/Projects/Pteropod_IBM/Shelled_Pteropod_IBM/")
import spIBM

import project_funcs

In [4]:
logging.basicConfig(level=logging.DEBUG)

# Read in yaml file and user inputs

In [5]:
year, version, control, config_file = 1984, 2, 0, "IBM_config_parameters.yaml"
My_config = spIBM.read_config_files(config_file)
My_config.control = control
My_config.version = version

# Define fieldset

Defines the environment that the pteropods experience

In [6]:
fieldset = spIBM.read_environment(My_config, year, control)

INFO:root:Adding lower and upper bounds...


# Calculate initial population

We implemented boolean flags to check if the initial population needs to be calculated in case this has already been done

In [7]:
flag_calculate_initial_population = My_config.flag_calculate_initial_population

directory_mort = My_config.directory_mort
similarity_file = My_config.similarity_file
output_dir = My_config.output_dir_initialization

if not os.path.exists(output_dir):
    os.makedirs(output_dir)


For reproducibility, we use a predefined seed

In [8]:
np.random.seed(seed=My_config.seed)
my_pteropods = spIBM.define_initial_population(number_of_individuals=1500, start_generation=0,
                                               number_of_attributes=17)

Run the idealized version of the IBM, to produce a starting population.

In [9]:
spIBM.run_ibm_idealized(My_config, my_pteropods, start_gen=0, time=5000, length_t=None,
                            save_population=True, save_abundance=True)

718.0 Individuals: 100%|██████████| 4999/4999 [11:03<00:00,  7.53it/s]  


# Determine the starting day

Determine which is the starting day from the idealized run above based on the best fit to abundance observations

In [10]:
ref_data_file ="/home/ursho/PhD/Projects/Pteropod_IBM/Data/MarEDat20120203Pteropods.nc"
daily_abundance_maredat, std_abundance_maredat = project_funcs.get_daily_maredat_obs(ref_data=ref_data_file)


In [11]:
output_dir = My_config.output_dir_initialization
gen0_file = My_config.gen0_file
gen1_file = My_config.gen1_file
observations = daily_abundance_maredat
observations_std = std_abundance_maredat

start_day = spIBM.determine_starting_day(output_dir, gen0_file, gen1_file, observations,
                                         observations_std, start=None)

INFO:root:Matching to observations
INFO:root:The following metrics were found:
INFO:root:3501, 0.91, 0.99, 15.5, 0.29
INFO:root:Start day is: 3501


In [12]:
initial_population = np.genfromtxt(output_dir+'/Pteropods_Day_{}.csv'.format(int(start_day)),delimiter=',')
num_init = initial_population.shape[0]


# Get initial positions

Determine where each individual is seeded at the beginning of the simulation. Here again we use a pre-defined seed (previous seed$\times$5)

In [13]:

grid_file = My_config.mesh_file
outfile = My_config.output_dir_initialization+My_config.initial_positions_file

np.random.seed(seed=My_config.version*5)

#On the first year, calculate the initial positions. Later read them from file
latlon_list = project_funcs.get_initial_positions(num=num_init,grid_file=grid_file,outfile=outfile)

    
latlon_list = np.genfromtxt(outfile, delimiter=',')

INFO:root:Searching for positions...


In [14]:
pclass = spIBM.PteropodParticle
ptero_pset = spIBM.initialize_particles(fieldset,pclass,initial_population,latlon_list)

# Run physics only for X days

Run the IBM with physics only to initialize the depth, ascent, descent timings and other attributes/experienced environmental conditions

In [15]:
kernel = ptero_pset.Kernel(spIBM.pteropod_kernel)

In [16]:
ptero_pset = spIBM.run_physics_only(My_config, ptero_pset, fieldset, kernel, year, 
                                        total_runtime=3, dt=1.0, outputdt=1.0)
    
#always read from file. On the first year 1984 calculate the value and then read from file
my_file = My_config.output_dir_physics+My_config.physics_only_file.format(My_config.version)
ptero_pset = ParticleSet.from_particlefile(fieldset=fieldset, pclass=pclass, filename=my_file,
                                           lonlatdepth_dtype=np.float32)


Physics only progress:   0%|          | 0/3 [00:00<?, ?it/s]INFO: Compiled PteropodParticlepteropod_kernel ==> /tmp/parcels-43802/f17427094c066ca01e6f4900a4f06e32_0.so
INFO:parcels.tools.loggers:Compiled PteropodParticlepteropod_kernel ==> /tmp/parcels-43802/f17427094c066ca01e6f4900a4f06e32_0.so
Physics only progress: 100%|██████████| 3/3 [01:26<00:00, 28.82s/it]


Reset the time and attributes. Only the time should have changed by X days. Here we show how other attributes can be changed.

In [17]:
reset_dictionary = {'time': 0.0,
                    'MyID': initial_population[:,0],
                    'generation': initial_population[:,1],
                    'stage': initial_population[:,2],
                    'shell_size': initial_population[:,3],
                    'days_of_growth': initial_population[:,4],
                    'survive': initial_population[:,5],
                    'num_spawning_event': initial_population[:,6],
                    'ERR': initial_population[:,7],
                    'spawned': initial_population[:,8],
                    'Parent_ID': initial_population[:,9],
                    'Parent_shell_size': initial_population[:,10],
                    'damage': initial_population[:,14]}

ptero_pset = spIBM.reset_particle_attributes(ptero_pset,reset_dictionary)


# Start coupled run

In [22]:
print('Starting simulation...')

next_ID = max(initial_population[:,0])+1
print(f'Shape initial {initial_population.shape}')
print(f'Next ID is {next_ID}')
current_gen = np.nanmax(initial_population[np.squeeze(np.argwhere((initial_population[:,2]==3) | (initial_population[:,3] == max(np.unique(initial_population[:,3]))))).astype(int),1])


Starting simulation...
Shape initial (994, 17)
Next ID is 1365961.0


In [19]:
d0 = datetime.date(year,1,1)
d1 = datetime.date(year,1,10)
time_mat = np.empty((3,(d1-d0).days))
for i in range(time_mat.shape[1]):
    time_mat[0,i] = (d0+datetime.timedelta(days=i)).year
    time_mat[1,i] = (d0+datetime.timedelta(days=i)).day
    time_mat[2,i] = i

In [20]:
spIBM.run_ibm_coupled(My_config, ptero_pset, fieldset, pclass, kernel, time_mat, next_ID, 
                      current_gen, length_t=None)